In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [8]:
def get_top_stock():
    try:
        res=requests.get('https://cafef.vn/get-top-stock.chn')
        data=res.json()['stocks']
    except:
        return "Không thể tìm nạp dữ liệu"    
    return data 

def get_info_stock(stock_name):
    try:
        res=requests.get(f'https://e.cafef.vn/info.ashx?type=cp&symbol={stock_name}')
        data=res.json()
    except:
        return "Không thể tìm nạp dữ liệu hoặc mã cổ phiếu cần tìm bị sai"    
    return data 

In [9]:
def get_list_post(page_num):
    try:
        res=requests.get(f'https://cafef.vn/timelinelist/18831/{page_num}.chn')
        soup=BeautifulSoup(res.content,'html.parser')
        data=soup.find_all(class_='tlitem box-category-item')


    except Exception as e :
        return f"gặp lỗi : {e}"    
    return data 

In [10]:
def get_info__post(data):
    times=[]
    desc=[]
    links=[]
    titles=[]
    for i in range (len(data)):
        times.append(data[i].find(class_='time time-ago').text)
        desc.append(data[i].find(class_='sapo box-category-sapo').text)
        links.append('https://cafef.vn'+data[i].find('a',href=True)['href'])
        titles.append(data[i].find('a',href=True).text)

    return times,desc,links,titles

In [11]:
def get_content_post(link_post):
    content=''
    try:
        res=requests.get(link_post)
        soup=BeautifulSoup(res.content,'html.parser')
        c=soup.find(class_='detail-content afcbc-body')
        p=c.find_all('p')
        for i in range(len(p)):
            content=content+ p[i].text+ '\n'



    except Exception as e :
        return f"gặp lỗi : {e}"    
    return content 

In [12]:
column_names = ['Title', 'Desc', 'Content','Link','Time']  

df = pd.DataFrame(columns=column_names)

In [13]:
for i in range(1,15):
    content=[]
    times,desc,links,titles=get_info__post(get_list_post(i))
    for j in range(len(links)):
        content.append(get_content_post(links[j]))
       
    temp_df = pd.DataFrame({'Title':titles, 'Desc':desc, 'Content':content,'Link':links,'Time':times})
    df=pd.concat([df, temp_df])

In [16]:
df.head(10)

,Title,Desc,Content,Link,Time
0,Khối ngoại mua ròng HPG 15 phiên liên tiếp,HPG cùng với một số cổ phiếu ngân hàng là nhữn...,Khối ngoại mua ròng HPG 15 phiên liên tiếp. Ản...,https://cafef.vn/khoi-ngoai-mua-rong-hpg-15-ph...,2023-06-29T08:17:00
1,Lịch sự kiện và tin vắn chứng khoán ngày 29/6,Tổng hợp toàn bộ tin vắn nổi bật liên quan đến...,Tin doanh nghiệp\nDPM - Tổng công ty Phân bón ...,https://cafef.vn/lich-su-kien-va-tin-van-chung...,2023-06-29T07:36:00
2,"Bị kiểm toán nhấn mạnh khoản ""giấu lỗ"" HPG, cô...","Đồng thời, kiểm toán còn đưa ra ý kiến ngoại t...","Theo thông báo mới nhất, CTCP Chứng khoá...",https://cafef.vn/bi-kiem-toan-nhan-manh-khoan-...,2023-06-29T00:06:00
3,MBS: VN-Index khó bùng nổ nếu các nút thắt chư...,Thị trường sẽ khó có sự bùng nổ về điểm số và ...,"Trong báo cáo chiến lược mới công bố, Chứng kh...",https://cafef.vn/mbs-vn-index-kho-bung-no-neu-...,2023-06-29T00:03:00
4,ĐHĐCĐ IPA: Đặt kế hoạch doanh thu và lợi nhuận...,Ông Mai Hữu Đạt - Tổng Giám đốc IPA cho biết d...,Đặt kế hoạch doanh thu và lợi nhuận “tăng ...,https://cafef.vn/dhdcd-ipa-dat-ke-hoach-doanh-...,2023-06-29T00:02:00
5,Khối ngoại bán ròng Vinamilk 5 tháng liên tiếp...,"Nếu chỉ tính riêng khớp lệnh, Vinamilk là cổ p...",Sau một thời gian dài gần như “im hơi lặng t...,https://cafef.vn/khoi-ngoai-ban-rong-vinamilk-...,2023-06-29T00:01:00
6,Ca sĩ “Chiếc khăn gió ấm” Khánh Phương bị phạt...,"Trong một chia sẻ mới đây, nam ca sĩ “Chiếc kh...",Ủy ban Chứng khoán Nhà nước vừa ban hành Quy...,https://cafef.vn/ca-si-chiec-khan-gio-am-khanh...,2023-06-28T22:26:00
7,Egroup của Shark Thuỷ tiếp tục bị công ty chứn...,"Tạm tính theo thị giá bình quân, lượng cổ phiế...","Trong thông báo mới nhất, Tập đoàn Giáo dục ...",https://cafef.vn/egroup-cua-shark-thuy-tiep-tu...,2023-06-28T19:14:00
8,"Cổ phiếu Novaland sang tay nghìn tỷ, nhóm Apec...",VN-Index có phiên thứ 7 tăng giá liên tiếp. Nh...,"Thị trường lại khởi sắc về cuối phiên, khi n...",https://cafef.vn/co-phieu-novaland-sang-tay-ng...,2023-06-28T19:11:00
9,"Góc nhìn CTCK: Tiếp tục xu hướng tăng, tập tru...","Nhận định thị trường phiên mai, Agriseco Resea...",\nVN-Index tiếp chuỗi 7 phiên tăng điểm liên t...,https://cafef.vn/goc-nhin-ctck-tiep-tuc-xu-huo...,2023-06-28T19:10:00


In [15]:
df.to_csv('dulieu.csv')